### This notebook consist of code for creating the html files for the website each time data is updated.

##### Set-up

In [597]:
#Importing classes from the Jinja2 library to load and render templates.
import json
import os
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError
from pathlib import Path


In [598]:
MEMBERS_DIR_PATH = Path('/Users/harshul/website clone/harshul/test/group-data/members/')


In [599]:
#Function for creating proper html file names
def page_link(a):
    if ' ' in a:
        return a.replace(' ', '_')
    else:
        return a

In [600]:
member_records = []

for member_dir in MEMBERS_DIR_PATH.glob('*'):
   
    if member_dir.name.startswith('.'):
        continue
    
    member_record = json.load(open(member_dir / 'info.json'))
    member_json_dir = member_dir / 'jsons'
    
    if (member_experiences := (member_json_dir / 'experiences.json')).exists():
        current_position = json.load(open(member_experiences))[0]
        member_record.update(current_position)
        
    if (member_education := (member_json_dir / 'education.json')).exists():
        current_position = json.load(open(member_education))[0]
        member_record.update(current_position)

    if (member_projects := (member_json_dir / 'projects.json')).exists():
        current_position = json.load(open(member_education))[0]
        member_record.update(current_position)
    
    if (member_links := (member_json_dir / 'social_links.json')).exists():
        social_links = json.load(open(member_links))
        member_record.update(social_links)
    
    member_records.append(member_record)


In [601]:
people_page_df=pd.DataFrame.from_records(member_records)


In [602]:
people_page_df

,first_name,last_name,nick_name,image_path,cover_image_path,introduction,id,institution,city,state,...,subject,project_title,description,website,github_handle,twitter_handle,linkedin_handle,email,orcid,start-date
0,Joshua,Shields,Josh,media/images/josh_photo.jpg,media/images/cover.jpg,Josh is a senior graduate student in astrophys...,josh_shields,Michigan State University,East Lansing,Michigan,...,Physics and Astronomy.,Surviving Companions Of Supernovae And Stellar...,,https://jvshields.github.io/,jvshields,,,shield90@msu.edu,0000-0002-1560-5286,NaN
1,Anirban,Dutta,,media/images/anirban_dutta.jpg,media/images/cover.jpg,Hi there! This is Anirban.,anirban_dutta,Indian Institute of Astrophysics,Bangalore,Karnataka,...,Astrophysics,Non-LTE modeling of supernova spectra,,https://sites.google.com/view/anirbaniamdutta,Knights-Templars,Anirban29Dutta,anirban-dutta-6a0377238,anirbaniamdutta@gmail.com,0000-0002-7708-3831,NaN
2,Erin,Visser,,media/images/erin_visser_website_pic.jpg,media/images/cover.jpg,,erin_visser,Michigan State University,East Lansing,Michigan,...,"Physics and Mathematics, Advanced",NaN,NaN,,erinvisser,,,visserer@msu.edu,0009-0001-8470-275X,NaN
3,Vicente,Amado Olivo,,media/images/ESD_headshot.jpg,media/images/cover.jpg,I am a graduate student at Michigan State Univ...,vicente_amado,Michigan State University,East Lansing,Michigan,...,Finance with a minor in Computational Mathemat...,Development Of A Global Registry For Peer Revi...,,,,vamadolivo,,amadovic@msu.edu,0000-0003-2248-0941,NaN
4,Yuki,Matsumura,,media/images/yuki_face.png,media/images/cover.jpg,"Hello, I am a first-year physics graduate stud...",yuki_matsumura,Michigan State University,East Lansing,Michigan,...,Physics and CMSE,Type IIP Supernovae As Cosmological Distance P...,,,ymatsumu,,,matsumurayuki725@gmail.com,,NaN
5,Andrew,Fullard,,media/images/me.jpg,media/images/cover.jpg,I joined the TARDIS group in 2020 and worked o...,andrew_fullard,University of Denver,Denver,Colorado,...,Astronomy,Inferring explosion conditions from late-time ...,,NaN,andrewfullard,astrofullard,andrew-fullard-a9a487168/,fullarda@msu.edu,0000-0001-7343-1678,NaN
6,Isaac,Smith,,media/images/isaac_image.jpg,media/images/cover.jpg,,isaac_smith,Michigan State University,East Lansing,Michigan,...,Physics and Mathematics,NaN,NaN,,smithis7,,,smithis7@msu.edu,0000-0003-0440-3918,NaN
7,Hayden,Monk,,media/images/hayden.jpg,media/images/cover.jpg,,hayden_monk,Michigan State University,East Lansing,Michigan,...,Astrophysics,Surviving Companion Search In SNR-0509,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Atharva,Arya,,media/images/atharva.jpg,media/images/cover.jpg,I joined TARDIS as a GSoC 2021 student and I h...,atharva_arya,RCOEM,Nagpur,Maharashtra,...,Engineering,Mitigating open science sustainability issues,,https://www.atharvaarya.tech/,atharva-2001,2001_atharva,atharva-arya,aryaatharva18@gmail.com,,NaN
9,Richard,Dow,Tripp,media/images/richard.jpg,media/images/cover.jpg,"Hi, my name is Tripp, and I'm a computer scien...",richard_dow,University of Minnesota,Twin Cities,Minnesota,...,Computer Science,Identifying Worldwide Astrophysicists From Sci...,,,prettytrippy,,,dow00019@umn.edu,0009-0007-2842-1690,NaN


In [603]:
WEBSITE_DATA_PATH = Path('../group-data/website_data/')


website_data_files = {
    'contact': 'contact.json',
    'general': 'general.json',
    'research': 'research.json',
    'homepage': 'homepage.json',
    'support': 'support.json'
}


contact, general, research, homepage, support = [{}] * 5


for var_name, file_name in website_data_files.items():
    file_path = WEBSITE_DATA_PATH / file_name
    if file_path.exists():
        with open(file_path, 'r') as f:
            locals()[var_name] = json.load(f)


In [604]:
contact_df = pd.DataFrame.from_records([contact])
general_df = pd.DataFrame.from_records([general])
homepage_df = pd.DataFrame.from_records([homepage])
research_df = pd.DataFrame.from_records([research])
support_df = pd.DataFrame.from_records([support])


In [605]:
content_dir_path = Path('../group-data/website_data/content/')
content_dfs = []  
for json_file in content_dir_path.glob('*.json'):
    with open(json_file, 'r') as file:
        data = json.load(file)
        
       
        df = pd.json_normalize(data)
        
        content_dfs.append(df)


content_df = pd.concat(content_dfs, ignore_index=True, sort=False)




In [606]:
#Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates
environment = Environment(loader=FileSystemLoader('templates/'),extensions=['jinja2.ext.loopcontrols'])
environment.globals['page_link'] = page_link
environment.globals['ContentData'] = content_df

In [607]:
research_content_unsorted = content_df[content_df['tags'].apply(lambda x: any('research' in tag for tag in x))]
research_content = research_content_unsorted.groupby('category').apply(lambda x: x.sort_values('article_date', ascending=False)).reset_index(drop=True)

In [608]:
news_content_unsorted = content_df[content_df['tags'].apply(lambda x: any('news' in tag for tag in x))]
news_content = news_content_unsorted.sort_values(by="article_date", ascending=False)

In [609]:
latest_content_df = pd.DataFrame()

for category in content_df.category.unique():
    latest_data = pd.Series(content_df[content_df.category == category].iloc[0])
    latest_content_df = latest_content_df._append(latest_data, ignore_index=True)

latest_content_df['article_date'] = pd.to_datetime(latest_content_df['article_date'], format='%m-%d-%Y')
latest_content_df = latest_content_df.sort_values(by='article_date', ascending=False)

##### Homepage

In [610]:
homepage_template = environment.get_template('homepage.html.j2')

In [611]:
general_dict = general_df.iloc[0].to_dict()
homepage_dict = homepage_df.iloc[0].to_dict()


In [612]:
homepage_content = homepage_template.render(general=general_dict,
                                            homepage=homepage_dict,
                                            recent_content=latest_content_df.to_dict(orient='records'))


TypeError: 'DataFrame' object is not callable

In [ ]:
with open('../kerzendorf-group.github.io/index.html', mode='w', encoding='utf-8') as Homepage:
    Homepage.write(homepage_content)

##### People Page

In [613]:
people_template = environment.get_template("people.html.j2")

In [614]:
people_content = people_template.render(general=general_dict, members=people_page_df)

In [615]:
with open("../kerzendorf-group.github.io/People.html", mode="w", encoding="utf-8") as people:
    people.write(people_content)

##### Individual People Page

In [ ]:
ind_person_template = environment.get_template("individual_person.html.j2")

In [ ]:
for person in people_df['id']:
            filename = f"../kerzendorf-group.github.io/members/{ person }/{ person }.html"
            ind_person_content = ind_person_template.render(general=data["general"], 
                                                            member_id=person, 
                                                            content=content_df.to_dict(orient='records'))
            with open(filename, mode="w", encoding="utf-8") as page:
                page.write(ind_person_content)

##### Research Page

In [ ]:
research_template = environment.get_template("research.html.j2")

In [ ]:
main_page_research_content = research_template.render(general=data["general"],
                                            content=research_content)

In [ ]:
with open("../kerzendorf-group.github.io/Research.html", mode="w", encoding="utf-8") as research:
        research.write(main_page_research_content)

In [ ]:
sub_research_template = environment.get_template("sub_research_frontpage.html.j2")

In [ ]:
for category in content_df.loc[content_df.category != "News", "category"].unique():
        sub_research_content = sub_research_template.render(general=data["general"], 
                                                            research_general=data["research"], 
                                                            content = research_content,
                                                            category = category
                                                            )
        folder_path = f"../kerzendorf-group.github.io/sub_research/{page_link(category.lower())}"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"../kerzendorf-group.github.io/sub_research/{page_link(category.lower())}.html", mode="w", encoding="utf-8") as sub_research:
            sub_research.write(sub_research_content)

##### Individual Research Page

In [ ]:
template_no_twitter = environment.get_template("research_page_no_twitter.html.j2")

In [ ]:
for ind_research_keys, ind_research_values in research_content.iterrows():
    if "news" not in ind_research_values.category.lower():
        ind_research_content = template_no_twitter.render(general=data["general"], 
                                                          member_ids = people_df['id'],
                                                          nonmem_ids = non_mem_df['id'],
                                                          content = ind_research_values
                                                          
                                                            )
        folder_path = f"../kerzendorf-group.github.io/sub_research/{page_link(ind_research_values.category.lower())}"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"{ folder_path }/{page_link(ind_research_values.article_id.lower())}.html", mode="w", encoding="utf-8") as ind_research_page:
            ind_research_page.write(ind_research_content)

##### News Page

In [ ]:
news_content

In [ ]:
news_template = environment.get_template("news.html.j2")

In [ ]:
news_page_content = news_template.render(general=data["general"],
                                         content=news_content,
                                         member_ids=people_df['id'],
                                         nonmem_ids = non_mem_df['id'],
                                         category="News")

In [ ]:
with open("../kerzendorf-group.github.io/News.html", mode="w", encoding="utf-8") as news:
        news.write(news_page_content)

##### Individual News Pages

In [ ]:
news_template_no_twitter = environment.get_template("news_page_no_twitter.html.j2")
#news_template_twitter = environment.get_template("news_page_twitter.html.j2")

In [ ]:
for ind_news_keys, ind_news_values in news_content.iterrows():
        ind_news_content = news_template_no_twitter.render(general=data["general"], 
                                                          member_ids = people_df['id'],
                                                          nonmem_ids = non_mem_df['id'],
                                                          content = ind_news_values
                                                            )
        folder_path = f"../kerzendorf-group.github.io/news/"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"{ folder_path }/{page_link(ind_news_values.article_id.lower())}.html", mode="w", encoding="utf-8") as ind_news_page:
            ind_news_page.write(ind_news_content)

##### Support Page

In [ ]:
support_template = environment.get_template('support.html.j2')

In [ ]:
support_content = support_template.render(general=data["general"], support=data["support"])

In [ ]:
with open('../kerzendorf-group.github.io/Support.html', mode='w', encoding='utf-8') as support:
    support.write(support_content)

##### Contact

In [ ]:
contact_template = environment.get_template('contact.html.j2')

In [ ]:
contact_content = contact_template.render(general=data["general"], contact=data["contact"])

In [ ]:
with open('../kerzendorf-group.github.io/Contact.html', mode='w', encoding='utf-8') as contact:
    contact.write(contact_content)